In [1]:
import pandas as pd
import numpy as np
from jqdatasdk import *
auth('15566918488','Shuiguo123')

提示：当前环境pandas版本为0.25，get_price与get_fundamentals_continuously接口panel参数将固定为False
注意：0.25以上版本pandas不支持panel，如使用该数据结构和相关函数请注意修改
auth success 


In [2]:
today = '2020-09-18'


In [28]:
df = pd.read_excel('/Users/kayamayajin/Desktop/draft1.xlsx',converters = {u'name':str,u'code':str,u'CODE':str,u'industry':str})
a = df.columns.tolist()
a.insert(0,'date')
df = df.reindex(columns = a)
df['date'] = df['date'].astype('str')
df['industry_code'] = None

df['date'] = today
df['CODE'] = df['code'].apply(lambda x: str(x)+'.XSHG' if int(x)>599999 else str(x)+'.XSHE')
df['name'] = df['CODE'].apply(lambda x: get_security_info(x).display_name)
df['industry'] = df['CODE'].apply(lambda x:get_industry(x,date = today)[x]['sw_l3']['industry_name'])
df['industry_code'] = df['CODE'].apply(lambda x:get_industry(x,date = today)[x]['sw_l3']['industry_code'])
df['market_cap'] =df['CODE'].apply(lambda x:get_fundamentals(query(valuation.market_cap).filter(valuation.code == x),today).iloc[0,0])
df


,date,code,CODE,name,market_cap,industry,industry_code
0,2020-09-18,300405,300405.XSHE,科隆股份,35.9955,其他化学制品III,850339
1,2020-09-18,300863,300863.XSHE,卡倍亿,71.5229,汽车零部件III,850921
2,2020-09-18,300569,300569.XSHE,天能重工,77.7072,风电设备III,857332
3,2020-09-18,600481,600481.XSHG,双良节能,73.2901,制冷空调设备III,850715
4,2020-09-18,300733,300733.XSHE,西菱动力,38.1280,汽车零部件III,850921
5,2020-09-18,603557,603557.XSHG,起步股份,59.9862,鞋帽III,851325
6,2020-09-18,300128,300128.XSHE,锦富技术,62.1458,显示器件III,850831
7,2020-09-18,300475,300475.XSHE,聚隆科技,56.3220,家电零部件III,851115
8,2020-09-18,600515,600515.XSHG,海航基础,405.9989,房地产开发III,851811
9,2020-09-18,002390,002390.XSHE,信邦制药,124.2085,中药III,851521


In [29]:
df.sort_values('industry_code', ascending = False)

,date,code,CODE,name,market_cap,industry,industry_code
2,2020-09-18,300569,300569.XSHE,天能重工,77.7072,风电设备III,857332
8,2020-09-18,600515,600515.XSHG,海航基础,405.9989,房地产开发III,851811
9,2020-09-18,002390,002390.XSHE,信邦制药,124.2085,中药III,851521
5,2020-09-18,603557,603557.XSHG,起步股份,59.9862,鞋帽III,851325
7,2020-09-18,300475,300475.XSHE,聚隆科技,56.3220,家电零部件III,851115
1,2020-09-18,300863,300863.XSHE,卡倍亿,71.5229,汽车零部件III,850921
4,2020-09-18,300733,300733.XSHE,西菱动力,38.1280,汽车零部件III,850921
6,2020-09-18,300128,300128.XSHE,锦富技术,62.1458,显示器件III,850831
3,2020-09-18,600481,600481.XSHG,双良节能,73.2901,制冷空调设备III,850715
0,2020-09-18,300405,300405.XSHE,科隆股份,35.9955,其他化学制品III,850339


In [27]:
df = pd.read_excel('/Users/kayamayajin/Desktop/各个指数.xlsx',sheet_name = '50')

In [26]:
#df.sort_values('industry_code', ascending = False)

In [24]:
#

df1 = pd.DataFrame(get_industry_stocks('857332',date=today))

def valuation_len(x):
    return len(get_fundamentals(query(valuation).filter(valuation.code == x),today))
    
def indicator_len(x):
    return len(get_fundamentals(query(indicator).filter(indicator.code == x),today))


##筛掉未上市的公司，未上市公司没有finance表
#df1['state'] = df1[0].apply(lambda x: len(get_fundamentals(query(finance.STK_LIST.state_id).filter(finance.STK_LIST.code==x),today)))
#df1 = df1[df1.state.isin([1])]###这个好慢
#df1 = df1.drop(df1[df1['state'] == 0].index)


##筛掉新股,新股没有indicator表
#df1['new'] = df1[0].apply(lambda x: len(get_fundamentals(query(indicator).filter(indicator.code == x),today)))
#df1 = df1.drop(df1[df1['new'] == 0].index)


df1['date'] = today
df1['name'] = df1[0].apply(lambda x: get_security_info(x).display_name if get_security_info(x) is not None else '-')
df1['market_cap'] = df1[0].apply(lambda x: get_fundamentals(query(valuation.market_cap).filter(valuation.code == x),today)['market_cap'][0] if valuation_len(x) > 0 else '-')
df1['pe'] = df1[0].apply(lambda x: get_fundamentals(query(valuation.pe_ratio).filter(valuation.code == x),today).iloc[0,0] if valuation_len(x) > 0 else '-')
df1['roe'] = df1[0].apply(lambda x: get_fundamentals(query(indicator.roe).filter(valuation.code == x),today).iloc[0,0] if indicator_len(x) > 0 else '-')
df1
#df1.to_excel('/Users/kayamayajin/Desktop/output.xlsx')

,0,date,name,market_cap,pe,roe
0,002202.XSHE,2020-09-18,金风科技,436.8720,18.9929,1.21
1,002531.XSHE,2020-09-18,天顺风能,130.7579,13.6837,6.77
2,300129.XSHE,2020-09-18,泰胜风能,50.4846,23.4415,4.29
3,300185.XSHE,2020-09-18,通裕重工,165.0211,54.9342,2.26
4,300443.XSHE,2020-09-18,金雷股份,62.3233,20.5303,5.85
5,300569.XSHE,2020-09-18,天能重工,77.7072,22.7019,5.96
6,300772.XSHE,2020-09-18,运达股份,40.0080,32.3767,1.64
7,300850.XSHE,2020-09-18,新强联,59.3812,35.1037,-
8,600290.XSHG,2020-09-18,*ST华仪,11.2466,-0.3727,-0.78
9,600416.XSHG,2020-09-18,*ST湘电,165.6156,-14.0668,-4.68


In [31]:
#df1.to_excel('/Users/kayamayajin/Desktop/output.xlsx')

a = 'roe'

df1 = df1.drop(df1[df1[a] == '-' ].index)

df1.sort_values(a,ascending = False)

,0,date,name,market_cap,pe,roe
12,603218.XSHG,2020-09-18,日月股份,188.4434,27.0088,7.42
1,002531.XSHE,2020-09-18,天顺风能,130.7579,13.6837,6.77
5,300569.XSHE,2020-09-18,天能重工,77.7072,22.7019,5.96
4,300443.XSHE,2020-09-18,金雷股份,62.3233,20.5303,5.85
11,601615.XSHG,2020-09-18,明阳智能,225.7541,24.8259,5.19
2,300129.XSHE,2020-09-18,泰胜风能,50.4846,23.4415,4.29
10,601218.XSHG,2020-09-18,吉鑫科技,36.1992,25.1557,3.26
3,300185.XSHE,2020-09-18,通裕重工,165.0211,54.9342,2.26
13,603507.XSHG,2020-09-18,振江股份,35.4245,37.9025,2.25
6,300772.XSHE,2020-09-18,运达股份,40.0080,32.3767,1.64


In [79]:
get_security_info('IF2009.CCFX')
#df = get_extras('futures_sett_price', 'IF2009.CCFX', start_date='2020-08-25', end_date='2020-09-17', df=True)
df = get_price('IF2009.CCFX', start_date='2020-08-25', end_date='2020-09-17', frequency='daily', fields= None, skip_paused=False, fq='pre')
df.to_excel('/Users/kayamayajin/Desktop/output.xlsx')
print('搞定！')

搞定！
